## Packages and Functions Importation

In [3]:
import math 
import numpy as np
import tdmclient.notebook

import cv2
from ultralytics import YOLO

from CV_function import *
from path_object import *
from Markov_Filter import *

## Thymio Functions

In [ ]:
#Parameters 
WALL_THRESHOLD = 500
ROBOT_SPEED = 150
CM_SEC = 6 # 6 cm per second at motor speed 150 used for driving straigt
SEC_REVOLUTION = 9 # 9 seconds for 360 degrees at motor speed 100 used for turning

#Initialize robot
try:
    await tdmclient.notebook.stop()
except:
    pass
await tdmclient.notebook.start()

#function to sync variables between the notebook and the robot
@tdmclient.notebook.sync_var
def get_prox():
    global prox_horizontal
    return prox_horizontal

def local_navigation():
    global WALL_THRESHOLD, ROBOT_SPEED
    max_allowed_speed = 150
    set_var(leds_top = [32, 0, 0])
    while max(get_prox())>WALL_THRESHOLD:
        weight_left = [25,  15, -20, -15, -25]
        weight_right = [-25, -15, -15,  15,  25]
    
        # Scale factors for sensors and constant factor
        sensor_scale = 500
    
        mem_sensor = [0,0,0,0,0]
        prox_horizontal = get_prox()
        
        for i in range(len(mem_sensor)):
            # Get and scale inputs
            mem_sensor[i] = prox_horizontal[i] // sensor_scale
    
        y = [ROBOT_SPEED,ROBOT_SPEED]   
        
        for i in range(len(mem_sensor)):   
            # Compute outputs of neurons and set motor powers
            y[0] = y[0] + mem_sensor[i] * weight_left[i]
            y[1] = y[1] + mem_sensor[i] * weight_right[i]
    
        # Set motor powers
        set_var(motor_left_target = min(y[0],max_allowed_speed))
        set_var(motor_right_target = min(y[1],max_allowed_speed))
        time.sleep(0.2)
    set_var(leds_top = [0, 0, 32])

    return



@tdmclient.notebook.sync_var
# drive straight with "speed" (motor speed)
def motors_drive(l_speed=500, r_speed=500):
    global motor_left_target, motor_right_target
    
    motor_left_target = l_speed
    motor_right_target = r_speed

# drive "dist" [cm]
def drive(dist=10):
    global WALL_THRESHOLD, ROBOT_SPEED, CM_SEC
    motors_drive(ROBOT_SPEED,ROBOT_SPEED) #test with lower speed value
    for i in range(20):
        time.sleep((dist/(CM_SEC))/20) #motor speed = 150 equals 6cm/s -> calculates how many s you have to drive to cover a distance "dist" [cm]
        if max(get_prox()) > WALL_THRESHOLD:
            print("obstacle detected")
            local_navigation()
            break
    motors_drive(0,0)


@tdmclient.notebook.sync_var 
# turn in a circle with "speed" (motor speed)
def motors_turn(speed=100):
    global motor_left_target, motor_right_target
    motor_left_target = -speed
    motor_right_target = speed
    
@tdmclient.notebook.sync_var 
def motors_turn_right(speed=100):
    global motor_left_target, motor_right_target
    motor_left_target = speed
    motor_right_target = -speed

# turn "angle" [degrees] anticlockwise
def turn_degrees(angle):
    global SEC_REVOLUTION
    if angle < 0:
        motors_turn()
        time.sleep(abs(angle)*SEC_REVOLUTION/360) # 9s for 360° turn with motor speed = 100
        motors_turn(speed=0)
    else:
        motors_turn_right()
        time.sleep(abs(angle)*SEC_REVOLUTION/360)
        motors_turn_right(speed=0)

# turn "angle" (2. row of Astar_moves) then drive "dist" (1.row of Astar_moves)
def follow_path(dist,angle):
    turn_degrees(angle)
    drive(dist)
    
#used to initialize the follow robot command
def follow_path_init(path, current_angle,scale_factor):
    Ang = get_angle(path[0],path[1])
    Ang = Ang  - current_angle
    if abs(Ang) > 180:
                    if Ang > 0:
                        Ang = Ang-360
                    else:
                        Ang = Ang+360
    dist = get_distance(path[0],path[1])
    current_angle = Ang + current_angle
    follow_path(dist*scale_factor,-Ang)
    return current_angle



## Main loop

In [ ]:

map_max_x = 100
map_max_y = 75
#Initialize Global path object
global_nav = Global_nav()
#initialize markov filter map 
initialize_maps(map_max_x,map_max_y)


#Set current angle to 0
current_angle = 0
#robot LED in driving mode
set_var(leds_top = [0, 0, 32])

i = 0
#Initialize camera
cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID') 
output_ = cv2.VideoWriter('output.avi', fourcc, frames_per_second, (frame_height, frame_width)) 
while cap.isOpened():
    
    #image localization
    success, frame = cap.read()
    if  success:
        ## Computer vision detection
        array_box_dim_tot, annotated_frame = detection(frame)
        
        sorted_object = sorted(array_box_dim_tot, key=lambda x: order.get(x[0]))
        #break when q is pressed
        key = cv2.waitKey(1) & 0xFF 
        if key == ord('c'):  
            break  
    else:
        print("Unable to load the camera pic")
        break

    #fixing output from camera, use last output if no object is detected
    sorted_object = fix_output(sorted_object,sorted_object_last)
    

    #blind mode if no object is detected. Follow last calculated path with position and angle update from robot itself
    if sorted_object == False: 
        start = opti_path[1] 
        opti_path = opti_path[1:]
        if len(opti_path) == 1:
            print("Reached goal")
            set_var(leds_top = [0, 32, 0])
            break
        current_angle = follow_path_init(opti_path,current_angle,scale_factor)
        last_robot_pos = list(start)
        #print frame
        annotated_frame = frame
        cv2.imshow("frame", annotated_frame)
        continue #skip rest of loop to try to find object in next frame

    #save last output
    sorted_object_last = sorted_object

    #change format of pic output 
    start, goal, earth, robot_radius, earth_radius, black_holes_centers, black_holes_radiuss,scale_factor = global_nav.convert_OPENCV_tovalues(sorted_object)
    #if distance between robot and mars is less than 5 stop 
    if get_distance(start,goal) < 4:
        print("Reached goal")
        set_var(leds_top = [0, 32, 0])
        break
    
    #create map for gloabl navigation
    occ, _,_=  global_nav.create_map( goal = goal ,black_holes_centers = black_holes_centers,black_holes_radiuss=black_holes_radiuss)

    #use markov filter to update position 
    if i>0:
        start = markov(opti_path[1][0],opti_path[1][1],start[0],start[1],sorted_object[0][2],i==0)        

    #return path. Use the opti_path! 
    opti_path, _, _, _ = global_nav.get_path_straight(start=start,goal=goal)

    #convert optipath to format used by camera
    scaled_coords = convert_coordinates_2d_list(opti_path)

    #annotate frame
    if i>0:
        annotation(annotated_frame, scaled_coords)
    cv2.imshow("frame", annotated_frame)
    output_.write(annotated_frame)
    
    #update current angle. Take the average of the calculated from image and robot motion
    if i != 0: 
        if list(start) != last_robot_pos:
            current_angle = update_angle(last_robot_pos,start,current_angle)


    #follow first two points in path
    current_angle = follow_path_init(opti_path,current_angle,scale_factor)

    #save last robot position
    last_robot_pos = list(start)

    i+=1

#print last frame and save video
success, frame = cap.read()
array_box_dim_tot, annotated_frame = detection(frame)
cv2.imshow("frame", annotated_frame)
cv2.startWindowThread()
cap.release()
output_.release()
cv2.destroyAllWindows()